# AGENTE01 — Agente de IA para Perguntas e Respostas

Este agente tem como objetivo responder a perguntas feitas pelo usuário com base em um conteúdo pré-carregado (como arquivos `.txt`, `.pdf` ou outros documentos). 

A ideia central é criar uma interface simples e funcional para interagir com um LLM (Large Language Model), utilizando ferramentas como:

- **LangChain**: para orquestração do agente e carregamento dos dados
- **OpenAI API**: para gerar respostas com base nos dados
- **Chroma** ou outro vector store: para armazenar embeddings e fazer buscas semânticas

**Aplicações possíveis**:
- Consulta a manuais técnicos
- Resumo de relatórios
- Atendimento automático baseado em documentos

---

**Etapas que serão abordadas neste notebook**:
1. Instalação das bibliotecas
2. Carregamento dos documentos
3. Criação do index vetorial (embeddings)
4. Configuração do agente com LangChain
5. Execução de perguntas interativas


**Instalação das bibliotecas principais**;
1. pip install openai langchain chromadb tiktoken wheel

# Leitura de documentos (.txt, .pdf, .docx etc)
2. pip install unstructured pdfminer.six python-docx

# (Opcional) Biblioteca para carregar arquivos via widgets
3. pip install ipywidgets

# Se estiver usando dotenv para chaves (recomendado)
4. pip install python-dotenv

Obs.: A instalação foi concluída com sucesso, e esse aviso do langdetect é apenas uma mensagem de depreciação — não impede o funcionamento do agente.

**Carregamento de conteúdo fixo e criação dos embeddings**;
Essa célula:

Define um texto manual como base de conhecimento

Cria um documento no formato utilizado pelo LangChain

Gera os embeddings utilizando o modelo local sentence-transformers

Armazena os embeddings em uma base vetorial com o Chroma para consultas futuras

**Como ocultar o aviso (opcional)**;
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [1]:
"""
Célula 2: Bibliotecas principais do agente local
    Esta célula instala as bibliotecas necessárias para o agente local.
"""

executar_celula_5 = False

if executar_celula_5:
    # Bibliotecas principais do agente local
    !pip install -U langchain langchain-community chromadb tiktoken sentence-transformers transformers
    # Leitura de documentos (opcional)
    !pip install unstructured pdfminer.six python-docx
    # (Opcional) Interface interativa
    !pip install ipywidgets
    !pip install blobfile
    !pip install sentencepiece

else:
    # código da célula 5
    print("Execução interrompida da célula...")


Execução interrompida da célula...


In [2]:
"""
Célula 3: Carregamento de conteúdo fixo e criação dos embeddings locais

Esta célula:
- Define um texto manual como base de conhecimento
- Separa o texto em pequenos blocos (chunking)
- Gera embeddings com modelo local HuggingFace (sentence-transformers)
- Armazena os embeddings em uma base vetorial local usando Chroma
"""

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document


# Texto fixo como exemplo de base de conhecimento
texto = """
A inteligência artificial (IA) é o ramo da ciência da computação que se dedica a criar sistemas capazes de executar tarefas que normalmente exigiriam inteligência humana.
Essas tarefas incluem reconhecimento de fala, tomada de decisões, tradução de idiomas e muito mais.
Os agentes de IA utilizam modelos treinados em grandes quantidades de dados para realizar inferências e responder a comandos.
"""

# Separar o texto em blocos menores para melhor indexação
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=20)
documentos = text_splitter.split_documents([Document(page_content=texto)])

# Gerar embeddings locais com modelo Hugging Face
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Criar e armazenar os vetores em uma base vetorial local com Chroma
vectorstore = Chroma.from_documents(documentos, embedding=embedding)

C:\Users\carlo\AppData\Local\Temp\ipykernel_12196\460049780.py:29: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
"""
Célula 4: Execução do agente de perguntas e respostas

Esta célula:
- Recebe uma pergunta do usuário
- Usa o modelo local 'deepset/roberta-base-squad2' via Transformers
- Retorna a resposta com base no texto carregado
- Modelo com resposta a perguntas (QA) local, roberta-base-squad2, RESPOSTAS GENERICAS
"""

from transformers import pipeline

# Pipeline de QA
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

# Texto base como contexto
contexto = texto

# # Interação
# pergunta = input("Digite sua pergunta: ")
# resposta = qa_pipeline(question=pergunta, context=contexto)


# Pergunta simulada
pergunta = "Que tarefas a IA pode executar, segundo o texto?"
resposta = qa_pipeline(question=pergunta, context=contexto)


print("Pergunta:", pergunta)
print("Resposta:", resposta['answer'])


Device set to use cpu


Pergunta: Quais são os principais riscos éticos associados ao uso da inteligência artificial?
Resposta: muito mais


In [4]:
"""
Célula 5: Execução do agente de perguntas e respostas

Esta célula:
- Recebe uma pergunta do usuário
- Usa o modelo local 'distilbert-base-cased-distilled-squad' via Transformers
- Retorna a resposta com base no texto carregado
- Modelo com resposta a perguntas (QA) local, model="distilbert-base-cased-distilled-squad RESPOSTAS GENERICAS
"""

from transformers import pipeline

# Modelo alternativo mais objetivo
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

# Texto de contexto da célula 3
contexto = texto

# Pergunta simulada
pergunta = "Que tarefas a IA pode executar, segundo o texto?"

# Geração da resposta
resposta = qa_pipeline(question=pergunta, context=contexto)

print("Pergunta:", pergunta)
print("Resposta:", resposta['answer'])


Device set to use cpu


Pergunta: Que tarefas a IA pode executar, segundo o texto?
Resposta: normalmente exigiriam inteligência humana


In [5]:
"""
Célula 6: Execução do agente de perguntas e respostas

Esta célula:
- Recebe uma pergunta do usuário
- usar modelo generativo para perguntas e respostas
- Retorna a resposta com base no texto carregado
- Modelo com resposta a perguntas (QA) local, model=google/flan-t5-base RESPOSTAS MAIS COMPLETAS
- Usa o modelo google/flan-t5-base para gerar respostas completas com base no contexto.

Obs.: Treinado em inglês, mas pode responder em português
"""

from transformers import pipeline

# Pipeline de geração de texto com modelo T5
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")

# Montar prompt com contexto
prompt = f"Liste as tarefas que a inteligência artificial pode executar, segundo o texto: {texto}"

# Gerar resposta
resposta = qa_pipeline(prompt, max_length=60, do_sample=False)

print("Pergunta:", prompt)
print("Resposta:", resposta[0]['generated_text'])


Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Pergunta: Liste as tarefas que a inteligência artificial pode executar, segundo o texto: 
A inteligência artificial (IA) é o ramo da ciência da computação que se dedica a criar sistemas capazes de executar tarefas que normalmente exigiriam inteligência humana.
Essas tarefas incluem reconhecimento de fala, tomada de decisões, tradução de idiomas e muito mais.
Os agentes de IA utilizam modelos treinados em grandes quantidades de dados para realizar inferências e responder a comandos.

Resposta: List as tasks that artificial intelligence (AI) can execute, secondo o texto: Artificial intelligence (AI) é o ramo da ciência da computaço que se dedica a criar sistemas capazes de executar tasks that normally require human intelligence. These tasks include reconhecimento de fala, tomada decises, traducio de idiomas e muito mais. The agents of AI use three-dimensional models in large quantities of data to perform inferences and respond to commands.


In [ ]:
"""
Célula 7: Execução do agente com modelo generativo (google/mt5-small)

Esta célula:
- Recebe uma pergunta do usuário como prompt
- Utiliza um modelo generativo para responder com base no texto carregado
- Emprega o modelo local 'google/mt5-small', que oferece suporte multilíngue, incluindo o português
- Gera respostas mais completas e contextuais com base no conteúdo fornecido
"""

from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/mt5-small", use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

qa_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Reformulado com o prefixo "perguntar:", compatível com o estilo MT5
prompt = f"perguntar: Quais tarefas a inteligência artificial pode executar de acordo com o texto abaixo?\n{texto}"
# prompt = f"perguntar: Qual é a previsão do tempo para amanhã em São Paulo?\n{texto}"

# Lista de perguntas para testar o modelo
resposta = qa_pipeline(prompt, max_new_tokens=60, do_sample=False)

print(type(resposta))
print("Pergunta:", prompt)
print("Resposta:", resposta[0]['generated_text'])

Device set to use cpu


<class 'list'>
Pergunta: perguntar: Quais tarefas a inteligência artificial pode executar de acordo com o texto abaixo?

A inteligência artificial (IA) é o ramo da ciência da computação que se dedica a criar sistemas capazes de executar tarefas que normalmente exigiriam inteligência humana.
Essas tarefas incluem reconhecimento de fala, tomada de decisões, tradução de idiomas e muito mais.
Os agentes de IA utilizam modelos treinados em grandes quantidades de dados para realizar inferências e responder a comandos.

Resposta: <extra_id_0>.
